In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm.auto import tqdm
import concurrent.futures
from multiprocessing import Pool
import copy,os,sys
from collections import Counter,deque
import matplotlib.pyplot as plt
import json
import functools

In [2]:
import tensorflow as tf
import pandas as pd

# Test-Case

## load tfrecords

In [41]:
baseDir = "/tmp-data/zhoutongzt/Data/testData/BR"
baseDir = "/tmp-data/zhoutongzt/Data/trainData/BR"
trd_fp = [os.path.join(baseDir,i) for i in os.listdir(baseDir)]

config_path = open('/tmp-data/zhoutongzt/DD/tensorflow_lr/src/main/resources/features.json','r')
conf = json.load(config_path)
deepFeat = conf["deepFeat"]
varlensFeat_length = conf["varlensFeat_length"]
floatFeat = conf["floatFeat"]

if "testData" in baseDir:
    features = {
        "order_info":tf.io.FixedLenFeature([], tf.string),
        "wideFeatures":tf.io.FixedLenFeature([32], tf.int64),
        "label": tf.io.FixedLenFeature([], tf.int64)
    }
else:
    features = {
        "wideFeatures":tf.io.FixedLenFeature([32], tf.int64),
        "label": tf.io.FixedLenFeature([], tf.int64)
    }

for deepName in deepFeat:
    if deepName in varlensFeat_length:
        if deepName in floatFeat:
            features[deepName] = tf.io.FixedLenFeature([varlensFeat_length[deepName]], tf.float32)
        else:
            features[deepName] = tf.io.FixedLenFeature([varlensFeat_length[deepName]], tf.int64)
    else:
        if deepName in floatFeat:
            features[deepName] = tf.io.FixedLenFeature([], tf.float32)
        else:
            features[deepName] = tf.io.FixedLenFeature([], tf.int64)

def _parse_func(inp):
    return tf.io.parse_single_example(inp,features)




trd=tf.data.TFRecordDataset(trd_fp,compression_type='GZIP').map(_parse_func)
dataset=trd.shuffle(5*128).batch(2,drop_remainder=True)

In [42]:
with tf.Session() as sess:
    data = []
    for _ in range(10):
        line = dataset.make_one_shot_iterator().get_next()
        res = sess.run(line)
        if "testData" in baseDir:
            data.append(dict([(i,res[i].tolist()) for i in deepFeat+["order_info"]]))
        else:
            data.append(dict([(i,res[i].tolist()) for i in deepFeat]))

pd.DataFrame(data)

,dnnDGlobalTime,dnnNpoiDis,dnnPGlobalTime,dnncCurrentDis,dnncPoiDis,dnncRgeoDis,dnnpidDropoffHeat,dnnpidDropoffRatio,dnnpidLBChargeTime,dnnpidLFChargeTime,...,dnnpoiDropoffRatio,dnnpoiRatio,dnnpsw,link,locHash,pickupHash,pid,poi,rgeoHash,wifi
0,"[195266, 278046]","[0.639999985695, 0.560000002384]","[195175, 195175]","[20.9099998474, 7.07000017166]","[15.3500003815, 17.9699993134]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[14.75, 15.1499996185]","[47.6500015259, 31.25]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1..."
1,"[277955, 195266]","[0.0, 0.0]","[195158, 195175]","[68.1200027466, 68.6699981689]","[0.0, 55.0499992371]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[8.02000045776, 0.0]","[36335, -1]","[-1, -1]","[169153, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[[46174, 11226, 29051, -1, -1, -1, -1, -1], [-..."
2,"[195266, 195266]","[0.119999997318, 0.889999985695]","[195175, 195175]","[66.0400009155, 57.1100006104]","[39.6199989319, 69.4400024414]","[0.0, 0.0]","[0.0, 6.0]","[0.0, 1.77999997139]","[0.0, 4.76000022888]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 2.5]","[0.0, 0.0]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1..."
3,"[195266, 195266]","[0.689999997616, 0.0]","[195175, -1]","[35.5200004578, 21.7999992371]","[35.5200004578, 0.0]","[0.0, 0.0]","[0.0, 1.0]","[0.0, 2.22000002861]","[0.0, 0.0]","[0.0, 0.0]",...,"[3.57999992371, 0.0]","[3.0, 0.0]","[7.40000009537, 0.0]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1..."
4,"[195266, 195266]","[0.0, 0.0]","[195175, 195175]","[43.5499992371, 60.5400009155]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[26.0799999237, 0.0]","[69833, -1]","[-1, -1]","[222600, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1..."
5,"[1152, -1]","[0.670000016689, 1.0]","[195175, 195175]","[25.6599998474, 24.4400005341]","[13.2899999619, 15.8900003433]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[1.82000005245, 21.2900009155]","[123793, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1..."
6,"[195266, 195237]","[0.0, 0.0]","[195175, 195175]","[116.059997559, 3.90000009537]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[27.9599990845, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 39.8699989319]","[331763, 350274]","[-1, -1]","[-1, 320837]","[-1, 184439]","[-1, -1]","[-1, -1]","[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1..."
7,"[195266, 195266]","[0.930000007153, 0.0]","[195175, 195175]","[25.5599994659, 39.9399986267]","[22.3999996185, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[-1, -1]","[-1, -1]","[-1, -1]","[324148, -1]","[-1, -1]","[-1, -1]","[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1..."
8,"[195266, 195266]","[0.540000021458, 0.0]","[195175, 195175]","[126.989997864, 68.5800018311]","[26.3299999237, 70.1200027466]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1..."
9,"[195266, 195266]","[0.0, 0.0]","[195175, 195175]","[18.8600006104, 49.1500015259]","[11.1599998474, 0.0]","[0.0, 0.0]","[1.0, 0.0]","[10.0, 0.0]","[15.8500003815, 0.0]","[0.0, 0.0]",...,"[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[217607, 199469]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[-1, -1]","[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1..."


## load tfrecords (test/validation)

In [ ]:
testDir = "/home/zhoutongzt/zhoutongzt/DD/BR"

## load Parquet

In [38]:
onehot_fp = "/tmp-data/zhoutongzt/Data/dict_hashdemoB"
onehot_large_fp = "/tmp-data/zhoutongzt/Data/dict_hashdemoB_largeFeature"
onehot_deep_fp = "/tmp-data/zhoutongzt/Data/dict_hashdemoB_deepFeature"
hash_size = pd.read_parquet(onehot_fp).shape[0]
large_size = pd.read_parquet(onehot_large_fp).shape[0]
deep_size = pd.read_parquet(onehot_deep_fp).shape[0] # emb_size | 这里都是category特征

hash_size + large_size
deep_size

96134

224524

In [ ]:
fp = "/tmp-data/zhoutongzt/DD/Data/testData/BR_txt/part-00000-7e38355a-23dd-417b-b9fd-b00b016b9da1-c000.csv"

## Run Train

## Run Test

In [4]:
ckpDir = "/tmp-data/zhoutongzt/DD/Data/models/"
import sys
codeDir = "/tmp-data/zhoutongzt/DD/tensorflow_lr/"
sys.path.append(codeDir+"model_src/wide_and_deep")
sys.path.append(codeDir+"model_src/utils")
from conf_utils import DataConf
from wide_deep_train import WideAndDeep
import tfrecord_input

country_id="BR"
confInstance = DataConf.getInstance(codeDir+"conf/data.conf")
confInstance.init_fp
_, test_dir, model_dir = confInstance.getDir()
test_dir = os.path.join(test_dir, country_id)
model_dir = os.path.join(model_dir, country_id)
params = confInstance.getParams(country_id)
config_path = codeDir+'/src/main/resources/features.json'
wide_deep = WideAndDeep(params, config_path, model_dir)
classifier = wide_deep.build_estimator()



1.15.0
init_fp: /tmp-data/zhoutongzt/DD/tensorflow_lr/conf/data.conf


'/tmp-data/zhoutongzt/DD/tensorflow_lr/conf/data.conf'



Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': gpu_options {
  allow_growth: true
}
allow_soft_placement: true
, '_keep_checkpoint_max': 2, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa5f36d4d10>, '_model_dir': '/tmp-data/zhoutongzt/Data/models/BR', '_protocol': None, '_save_checkpoints_steps': 50000, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 5000, '_device_fn': None, '_session_creation_timeout_secs': 7200, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 10000, '_experimental_max_worker_delay_secs': None, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 

In [6]:
batch_size = wide_deep.batch_size
parallel_numbers = wide_deep.parallel_numbers

test_filenames_local = []
path_list = os.listdir(test_dir)
for file in path_list:
    if file.endswith('.gz'):
        filename_local = os.path.join(test_dir, file)
        test_filenames_local.append(filename_local)

test_inpf = functools.partial(tfrecord_input.eval_input_fn, test_filenames_local, batch_size, parallel_numbers)

### load trd

In [7]:
features = {
    "order_id":tf.io.FixedLenFeature([], tf.string),
    "distance":tf.io.FixedLenFeature([], tf.string)
}

def _parse_func(inp):
    return tf.io.parse_single_example(inp,features)


trd=tf.data.TFRecordDataset(test_filenames_local,compression_type='GZIP').map(_parse_func)
dataset=trd.batch(1,drop_remainder=False)
# trd_info = tf.data.TFRecordDataset(trd_fp,compression_type='GZIP').map(_parse_func)

In [9]:
line = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    for i in range(779139+1):
        res = sess.run(line)
        _ = res['order_id'],res['distance']
    print(i)

OutOfRangeError: End of sequence
	 [[node IteratorGetNext_1 (defined at /tmp-data/zhoutongzt/miniconda3/envs/py2.7/lib/python2.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for u'IteratorGetNext_1':
  File "/tmp-data/zhoutongzt/miniconda3/envs/py2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/tmp-data/zhoutongzt/miniconda3/envs/py2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/tmp-data/zhoutongzt/miniconda3/envs/py2.7/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/tmp-data/zhoutongzt/miniconda3/envs/py2.7/lib/python2.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/tmp-data/zhoutongzt/miniconda3/envs/py2.7/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/tmp-data/zhout

### predict 

In [11]:
help(tf.estimator.EvalSpec)

Help on class EvalSpec in module tensorflow_estimator.python.estimator.training:

class EvalSpec(EvalSpec)
 |  Configuration for the "eval" part for the `train_and_evaluate` call.
 |  
 |  `EvalSpec` combines details of evaluation of the trained model as well as its
 |  export. Evaluation consists of computing metrics to judge the performance of
 |  the trained model.  Export writes out the trained model on to external
 |  storage.
 |  
 |  Method resolution order:
 |      EvalSpec
 |      EvalSpec
 |      __builtin__.tuple
 |      __builtin__.object
 |  
 |  Static methods defined here:
 |  
 |  __new__(cls, input_fn, steps=100, name=None, hooks=None, exporters=None, start_delay_secs=120, throttle_secs=600)
 |      Creates a validated `EvalSpec` instance.
 |      
 |      Args:
 |        input_fn: A function that constructs the input data for evaluation.
 |          See [Premade Estimators](
 |          https://tensorflow.org/guide/premade_estimators#create_input_functions)
 |        

In [10]:
for i in dir(tf.estimator):
    if "Spec" in i:
        print(i)

EstimatorSpec
EvalSpec
TrainSpec


In [ ]:
with open(os.path.join(model_dir, 'checkpoint')) as f:
        next(f)
        for line in f:
            if line.split(':')[1].strip().strip('\"').split('-')[1] == '0':
                pass #continue
            ` = classifier.predict(test_inpf,checkpoint_path=os.path.join(model_dir,line.split(':')[1].strip().strip('\"')))
            pred = list(map(lambda x: (float(x['probabilities'][int(x['classes'][0])]), int(x['classes'][0])), [i for i in prediction]))
            
#             save_path = pred_path + str(count)
#             f = open(save_path, 'w')
#             f.write(str(pred))
            count += 1

it = trd_dataset_2_columns.make_oneshot_iterator()
with tf.Session() as sess:
    while True:
        line=sess.run(it.next())
    

### indicator

In [63]:
pred_file_name = "/tmp-data/zhoutongzt/DD/Data/testData/BR_pred1"
with open(pred_file_name,"r") as fr:
    preds = eval(fr.readlines()[0])


In [67]:
def func(ele):
    multi_pred = float(ele[0])
    multi_label = int(ele[1])
    order_info = ele[2]
    order_id, distance = order_info.split(';')[0].split("||")[0], float(order_info.split(';')[1])
    return [order_id, distance, multi_pred, multi_label]

In [68]:
pd.DataFrame(data=[func(p) for p in preds[:10]],columns=["order_id","distance","multi_pred","multi_label"])

,order_id,distance,multi_pred,multi_label
0,87969576360517||-43.34666_-21.79705||-43.36192...,121.814099,0.993432,0
1,87969579357352||-43.49019_-22.74938||-43.49001...,7.160207,0.690909,0
2,87969578929498||-46.56007_-23.66983||-46.55944...,93.917858,0.994952,0
3,87969580558722||-54.657_-20.51555||-54.6571090...,40.890767,0.530794,0
4,87969577989202||-46.78903_-23.58297||-46.78893...,64.759567,0.986107,0
5,87969570313973||-48.09427_-15.86986||-48.09420...,8.721107,0.288980,0
6,87969570792982||-46.54781_-23.69166||-46.54772...,12.768914,0.373720,2
7,87969577663841||-46.67274_-23.64598||-46.67324...,77.858039,0.694675,0
8,87969576962157||-46.46879_-23.54081||-46.46947...,70.380185,0.885555,0
9,87969577578169||-46.8695_-23.21615||-46.869466...,59.263371,0.991054,0
